## 모델 저장과 복원
---
- 방법1) 모델전체 저장(모델 구조 + 가중치 + 변수)
    * save_model()
    * 복원시 추가 작업 필요 없이 모델 파일 로딩해서 사용 가능

- 방법2) 가중치만 저장
    * save_weights()
    * 복원시 모델 구조(Architecture) 생성 후 가중치 적용
    
- 방법3) 모델 전체 또는 가중치 자동 저장
    * fit()에서 ModelCheckPoint Event에 대한 callback 등록
    * save_best_only=True : 모니터링 기준에 따라서 좋은 성능의 모델만 저장
    * save_weight_only=True : 가중치만 저장
    
- 파일 또는 폴더
    * 파일 확장자가 없으면 폴더로 저장
    * 파일 확장자
        - h5 / hdf5 : HDF5 포맷으로 모델 또는 가중치 저장
        - ckpf : 체크파일형태로 저장
        - pd : 모델 저장

## [1] 데이터 준비

In [575]:
from sklearn.datasets import load_iris
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import set_random_seed, plot_model
import numpy as np

## [1] 데이터 준비 및 로딩
---

In [576]:
iris=load_iris()

In [577]:
x_train=iris.data
y_train=iris.target

print(f'x_train.shape : {x_train.shape}, y_train.shape : {y_train.shape}')

x_train.shape : (150, 4), y_train.shape : (150,)


In [578]:
iris.target_names, iris.feature_names

(array(['setosa', 'versicolor', 'virginica'], dtype='<U10'),
 ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'])

## [2] 데이터 전처리
---

In [579]:
# Feature => 꽃잎의 길이, 너비
x_train=x_train[:, 2:]
x_train.shape

(150, 2)

In [580]:
# Target => 문자 => 수치화
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [581]:
lencoder=LabelEncoder()
y_train=lencoder.fit(y_train).transform(y_train)

In [582]:
ohencoder=OneHotEncoder()
y_train=ohencoder.fit(y_train.reshape(-1,1)).transform(y_train.reshape(-1,1))

In [583]:
y_train=y_train.toarray()

In [584]:
x_train.shape, y_train.shape, y_train[0]

((150, 2), (150, 3), array([1., 0., 0.]))

In [585]:
from sklearn.preprocessing import StandardScaler

In [586]:
scaler=StandardScaler()

In [587]:
x_scaled_train=scaler.fit(x_train).transform(x_train)

In [588]:
from sklearn.model_selection import train_test_split

In [589]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled_train,
                                                    y_train,
                                                    stratify=y_train)

In [590]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [591]:
print(f'x_train.shape : {x_train.shape}')
print(f'x_test.shape : {x_test.shape}')
print(f'x_val.shape : {x_val.shape}')

x_train.shape : (84, 2)
x_test.shape : (38, 2)
x_val.shape : (28, 2)


In [592]:
x_train[0]

array([-1.34022653, -1.3154443 ])

## [3] 모델 구성 및 생성
---
- 입력 데이터 : 2개 Feature
- 출력 데이터 : 3개 품종
- 학습 방법 : 분류 - 다중분류

In [593]:
# W,b 초기값 고정
set_random_seed(15)

In [594]:
irisModel=Sequential(name='irisModel')

In [595]:
irisModel.add(Dense(9, activation='relu', input_shape=(2,)))
irisModel.add(Dense(3, activation='softmax'))

In [596]:
irisModel.summary()

Model: "irisModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 9)                 27        
                                                                 
 dense_39 (Dense)            (None, 3)                 30        
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [597]:
# plot_model(irisModel, show_shapes=True)

In [598]:
## 모델 생성
irisModel.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics='accuracy')

## [4] 모델 학습

In [599]:
irisModel.fit(x_train, y_train, epochs=1000, verbose=False)

## [5] 평가

In [600]:
lossv, accv=irisModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.9737


In [601]:
lossv, accv

(0.061490535736083984, 0.9736841917037964)

## [6] 모델 저장

In [602]:
from tensorflow.keras.models import save_model

In [603]:
M_FILE='irisModel.h5'
W_FILE='irisWeight.h5'

In [604]:
if accv>=0.97:
    save_model(irisModel, M_FILE)
    irisModel.save_weights(W_FILE)

## [7] 복원
---
- 모델 또는 가중치 복원
- load_model(모델 파일 또는 모델 폴더명)
- Sequential.load_weights(가중치 파일 또는 폴더명)

### [7-1] 모델 복원

In [605]:
from tensorflow.keras.models import load_model

In [606]:
reModel=load_model(M_FILE)

In [607]:
reModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.9737


[0.061490535736083984, 0.9736841917037964]

In [608]:
reModel.summary()

Model: "irisModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 9)                 27        
                                                                 
 dense_39 (Dense)            (None, 3)                 30        
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


### [7-2] 가중치 복원
---
- 모델 구조 생성
- 가중치 로딩

In [609]:
# 모델 구조 구성
newModel=Sequential(name='newirisModel')
newModel.add(Dense(9, activation='relu', input_shape=(2,)))
newModel.add(Dense(3, activation='softmax'))
newModel.summary()

Model: "newirisModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 9)                 27        
                                                                 
 dense_41 (Dense)            (None, 3)                 30        
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [610]:
# 모델 생성
newModel.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics='accuracy')

In [611]:
# 가중치 로딩
newModel.load_weights(W_FILE)

In [612]:
newModel.evaluate(x_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.0615 - accuracy: 0.9737


[0.061490535736083984, 0.9736841917037964]